In [1]:
from itertools import permutations
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import explained_variance_score,mean_absolute_error
from datetime import datetime
from sklearn.neighbors import KNeighborsRegressor
#Ran at 120 for random forest got to %50 explained variance score using the 100000 dataset.

In [2]:
df=pd.read_csv("CRBE Challenge.csv")
df2=pd.read_csv("CRBE Challenge.csv")
#120 random forrest with 100000 dataset 62 percent actually 6 percent

In [3]:
df=pd.concat([df,df2])

In [4]:
df=df[['Asset ID',
'Asset Type',
'Floor',
'Room',
'Installation Date',
'Manufacturer',
'Operational Time (hrs)',
'Work Orders',
'Last Serviced Date',
'Repairs'
]]

In [5]:
df=df.sort_values(by=["Repairs",'Installation Date',
 'Last Serviced Date',
 'Work Orders',
 'Operational Time (hrs)',
 'Manufacturer',
 'Asset Type',
 'Asset ID'])
#Works without the reset index
#.reset_index(drop=True)
# with %50 reset index with drop set to true sort_by .sort_values(by=["Repairs",'Installation Date','Last Serviced Date','Work Orders','Operational Time (hrs)','Manufacturer','Asset Type']

In [6]:
#df=df.drop(columns="Asset ID")
df["Asset ID"]=df["Asset ID"].astype(str)

In [7]:
df["Floor"]=df["Floor"].astype(str)

In [8]:
df["Room"]=df["Room"].astype(str)

In [9]:
instances=[]
for repair_num in df["Repairs"]:
    if repair_num>df["Repairs"].quantile(0.75):
        instances.append("Extreme")
    else:
        instances.append("Not Extreme")
df["extreme_repair"]=instances

In [10]:
df["Installation Date"]=pd.to_datetime(df["Installation Date"])
instal_date=[int(date.timestamp()) for date in df["Installation Date"]]
df["Installation Date"]=instal_date
    
df["Last Serviced Date"]=pd.to_datetime(df["Last Serviced Date"])
service_date=[int(date.timestamp()) for date in df["Last Serviced Date"]]
df["Last Serviced Date"]=service_date

In [11]:
season=[]
#https://www.geeksforgeeks.org/print-the-season-name-of-the-year-based-on-the-month-number/
season_months = [[12 , 1 , 2], [3 , 4 , 5],
             [6 , 7 , 8], [9 , 10 , 11]]
for month in pd.to_datetime(df["Installation Date"]):
    if int(month.month) in season_months[0] :
        season.append('winter')
    elif int(month.month) in season_months[1] :
        season.append("spring")
    elif int(month.month) in season_months[2] :
        season.append("summer")
    elif int(month.month) in season_months[3] :
        season.append("fall")
df["season"]=season

In [12]:
df=df[[
'extreme_repair',
'Operational Time (hrs)',
'Work Orders',
#'old_instances',
'Asset ID',
'Asset Type',
'Floor',
'Room',
'season',
'Manufacturer',
'Last Serviced Date',
'Installation Date',
'Repairs']]

In [13]:
def train_test(df):
    y=df.iloc[:,len(df.columns)-1].values
    x=df.iloc[:,0:len(df.columns)-2].values
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
    return x_train,x_test,y_train,y_test

In [14]:
df=pd.get_dummies(df)

In [15]:
df_repairs=df[["Repairs"]]

In [16]:
df=df.drop(columns=["Repairs"])

In [17]:
df["Repairs"]=df_repairs

In [18]:
df.to_csv("CRBE Challenge Clean Stacked Data.csv", index=False)

In [19]:
df=df.drop(columns=["Repairs"])

In [20]:
df.to_csv("CRBE Challenge Clean Stacked Data Prediction Set.csv", index=False)

In [21]:
data_sets=train_test(df)

In [22]:
regression=LinearRegression()
regression.fit(data_sets[0],data_sets[2])
y_pred_lr=regression.predict(data_sets[1])

print('Variance score (lr): %.2f', explained_variance_score(data_sets[3], y_pred_lr))

Variance score (lr): %.2f 0.8301154405892616
